In [ ]:
TE = joinpath(tempdir(), "GSEA.test")

if isdir(TE)

    rm(TE; recursive = true)

    println("Removed ", TE, ".")

end

mkdir(TE)

println("Made ", TE, ".")

---

In [ ]:
using OnePiece

using GSEA

In [ ]:
da = joinpath(@__DIR__, "data")

se = joinpath(da, "h.all.v7.1.symbols.gmt")

;

## select_set

In [ ]:
GSEA.select_set(OnePiece.io.gmt.read(se), 33, 36)

## read_set

In [ ]:
GSEA.read_set(se, Dict("mi" => 33, "ma" => 36))

## run_single_sample_gsea

In [ ]:
ou = joinpath(TE, "single_sample_gsea.tsv")

GSEA.run_single_sample_gsea(
    joinpath(da, "setting", "single_sample_gsea.json"),
    se,
    joinpath(da, "nmf_k9_w.tsv"),
    ou,
)

readdir(TE)

## run_pre_rank_gsea

In [ ]:
ou = joinpath(TE, "pre_rank_gsea")

mkdir(ou)

GSEA.run_pre_rank_gsea(
    joinpath(da, "setting", "pre_rank_gsea.json"),
    se,
    joinpath(da, "gene_score.tsv"),
    ou,
)

readdir(ou)

## run_standard_gsea

In [ ]:
ou = joinpath(TE, "standard_gsea")

mkdir(ou)

GSEA.run_standard_gsea(
    joinpath(da, "setting", "standard_gsea.json"),
    se,
    joinpath(da, "sample_value.tsv"),
    joinpath(da, "nmf_k9_w.tsv"),
    ou,
)

readdir(ou)

## convert_gct_and_cls

In [ ]:
di = joinpath(da, "sarcopenia")

gc = joinpath(
    di,
    "gse111016_allsamplescounts_htseqcov1_sss_forgeo.sarcopenia.vs.normal_counts_collapsed_to_symbols.gct",
)

cl = joinpath(di, "sarcopenia_binary.cls")

GSEA.convert_gct_and_cls(gc, cl, TE)

readdir(di)

## convert_gmt

In [ ]:
gm = joinpath(di, "c2.cp.wikipathways.v7.4.symbols.gmt")

js = joinpath(TE, "set_to_genes.json")

GSEA.convert_gmt(gm, js)

readdir(di)

---

In [ ]:
if isdir(TE)

    rm(TE; recursive = true)

    println("Removed ", TE, ".")

end